# Dependency

In [1]:
import h5py
import os

from tqdm import tqdm
from time import gmtime, strftime

import numpy as np
import math

from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_curve

import tensorflow as tf
from tensorflow.keras import layers,Model

##

from sklearn.model_selection import KFold

import gc

import time
from sklearn.model_selection import KFold

#import import_test_ETC as load_data
import import_test_atp as load_data
#import import_test_nine as load_data

2023-08-16 22:43:32.098692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 22:43:32.157200: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-16 22:43:32.451588: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-08-16 22:43:32.451630: W tensorflow/compiler/xla/stream_executor/platf

# PARAM

In [2]:
DATA_LABEL=load_data.data_label()


NUM_DEPENDENT =7
MAXSEQ = NUM_DEPENDENT*2+1

DATA_TYPE = "ProtTrans"
#"/BinaryMatrix" "/MMseqs2" "/ProtTrans"


NUM_FILTER = 256
NUM_HIDDEN = 1000#100
BATCH_SIZE  = 1024
WINDOW_SIZES = [4,6,8,10]




NUM_CLASSES = 2
CLASS_NAMES = ['Negative','Positive']

NUM_FEATURE = 1024
EPOCHS      = 20

K_Fold = 5
VALIDATION_MODE="independent"
#"independent" "cross"
DATASET="new388+41"
#"new388+41" "old227+17"
IMBALANCE="None"
#None
#SMOTE
#ADASYN
#RANDOM



#IMBLANCE="RANDOM"#"ADASYN" "RANDOM" "None" "SMOTE"
#SHUFFLE="Non-SHUFFLE"#Non-SHUFFLE

In [3]:
import datetime

write_data=[]
a=datetime.datetime.now()
write_data.append(time.ctime())
write_data.append(DATA_LABEL)
write_data.append(DATASET)
write_data.append(DATA_TYPE)
write_data.append(WINDOW_SIZES)
write_data.append(NUM_FILTER)
write_data.append(NUM_DEPENDENT)
write_data.append(VALIDATION_MODE)
write_data.append(IMBALANCE)

#write_data.append(IMBLANCE)
#write_data.append(SHUFFLE)

# Time_log

In [4]:
def time_log(message):
    print(message," : ",strftime("%Y-%m-%d %H:%M:%S", gmtime()))

# MCNN

In [5]:
class DeepScan(Model):

	def __init__(self,
	             input_shape=(1, MAXSEQ, NUM_FEATURE),
	             window_sizes=[1024],
	             num_filters=256,
	             num_hidden=1000):
		super(DeepScan, self).__init__()
		# Add input layer
		self.input_layer = tf.keras.Input(input_shape)
		self.window_sizes = window_sizes
		self.conv2d = []
		self.maxpool = []
		self.flatten = []
		for window_size in self.window_sizes:
			self.conv2d.append(
			 layers.Conv2D(filters=num_filters,
			               kernel_size=(1, window_size),
			               activation=tf.nn.relu,
			               padding='valid',
			               bias_initializer=tf.constant_initializer(0.1),
			               kernel_initializer=tf.keras.initializers.GlorotUniform()))
			self.maxpool.append(
			 layers.MaxPooling2D(pool_size=(1, MAXSEQ - window_size + 1),
			                     strides=(1, MAXSEQ),
			                     padding='valid'))
			self.flatten.append(layers.Flatten())
		self.dropout = layers.Dropout(rate=0.7)
		self.fc1 = layers.Dense(
		 num_hidden,
		 activation=tf.nn.relu,
		 bias_initializer=tf.constant_initializer(0.1),
		 kernel_initializer=tf.keras.initializers.GlorotUniform())
		self.fc2 = layers.Dense(NUM_CLASSES,
		                        activation='softmax',
		                        kernel_regularizer=tf.keras.regularizers.l2(1e-3))

		# Get output layer with `call` method
		self.out = self.call(self.input_layer)

	def call(self, x, training=False):
		_x = []
		for i in range(len(self.window_sizes)):
			x_conv = self.conv2d[i](x)
			x_maxp = self.maxpool[i](x_conv)
			x_flat = self.flatten[i](x_maxp)
			_x.append(x_flat)

		x = tf.concat(_x, 1)
		x = self.dropout(x, training=training)
		x = self.fc1(x)
		x = self.fc2(x)  #Best Threshold
		return x

# Main

In [6]:
# Example usage:
x_train, y_train,x_test, y_test = load_data.MCNN_data_load(DATA_TYPE,NUM_CLASSES,DATASET,NUM_DEPENDENT)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/D/jupyter/atp_binding/dataset/Series15/ProtTrans/new388+41/atp-388'

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
def IMBALANCE_HANDLING(IM_BALANCE,x_train,y_train):
    from imblearn.over_sampling import SMOTE,ADASYN,RandomOverSampler

    # 將 x_train 的形狀重新整形為二維
    x_train_2d = x_train.reshape(x_train.shape[0], -1)
    print(x_train_2d.shape)
    print(y_train.shape)
    #print(y_train.shape)
    # 創建 SMOTE 物件
    if IM_BALANCE=="SMOTE":
        imbalance = SMOTE(random_state=42)
    elif IM_BALANCE=="ADASYN":
        imbalance = ADASYN(random_state=42)
    elif IM_BALANCE=="RANDOM":
        imbalance = RandomOverSampler(random_state=42)
    else:
        return
    

    # 使用 fit_resample 進行過採樣
    x_train_resampled, y_train_resampled = imbalance.fit_resample(x_train_2d, y_train)

    # 將 x_train_resampled 的形狀恢復為四維
    x_train_resampled = x_train_resampled.reshape(x_train_resampled.shape[0], 1, MAXSEQ, NUM_FEATURE)

    print(x_train_resampled.shape)
    print(y_train_resampled.shape)

    x_train=x_train_resampled
    y_train=y_train_resampled
    
    del x_train_resampled
    del y_train_resampled
    del x_train_2d
    gc.collect()

    import tensorflow as tf
    y_train = tf.keras.utils.to_categorical(y_train,NUM_CLASSES)

    return x_train,y_train

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
def model_test(model, x_test, y_test):

    print(x_test.shape)
    pred_test = model.predict(x_test)
    fpr, tpr, thresholds = roc_curve(y_test[:,1], pred_test[:, 1])
    AUC = metrics.auc(fpr, tpr)
    #tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
    display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=AUC, estimator_name='mCNN')
    display.plot()
    

    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
    print(f'Best Threshold={thresholds[ix]}, G-Mean={gmeans[ix]}')
    threshold = thresholds[ix]

    y_pred = (pred_test[:, 1] >= threshold).astype(int)

    TN, FP, FN, TP =  metrics.confusion_matrix(y_test[0:][:,1], y_pred).ravel()

    Sens = TP/(TP+FN) if TP+FN > 0 else 0.0
    Spec = TN/(FP+TN) if FP+TN > 0 else 0.0
    Acc = (TP+TN)/(TP+FP+TN+FN)
    MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)) if TP+FP > 0 and FP+TN > 0 and TP+FN and TN+FN else 0.0
    F1 = 2*TP/(2*TP+FP+FN)
    print(f'TP={TP}, FP={FP}, TN={TN}, FN={FN}, Sens={Sens:.4f}, Spec={Spec:.4f}, Acc={Acc:.4f}, MCC={MCC:.4f}, AUC={AUC:.4f}\n')
    
    return TP,FP,TN,FN,Sens,Spec,Acc,MCC,AUC

In [ ]:
if(VALIDATION_MODE=="cross"):
	time_log("Start cross")
	
	kfold = KFold(n_splits = K_Fold, shuffle = True, random_state = 2)
	results=[]
	i=1
	for train_index, test_index in kfold.split(x_train):
		print(i,"/",K_Fold,'\n')
		# 取得訓練和測試數據
		X_train, X_test = x_train[train_index], x_train[test_index]
		Y_train, Y_test = y_train[train_index], y_train[test_index]
		if IMBALANCE!="None":
				X_train,Y_train=IMBALANCE_HANDLING(IMBALANCE,X_train,Y_train)
		print(X_train.shape)
		print(X_test.shape)
		print(Y_train.shape)
		print(Y_test.shape)
		# 重新建模
		model = DeepScan(
		num_filters=NUM_FILTER,
			num_hidden=NUM_HIDDEN,
			window_sizes=WINDOW_SIZES)
		model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
		model.build(input_shape=X_train.shape)
		# 在測試數據上評估模型
		history=model.fit(
			X_train,
			Y_train,
			batch_size=BATCH_SIZE,
			epochs=EPOCHS,
			callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)],
			verbose=1,
			shuffle=True
		)
		TP, FP, TN, FN, Sens, Spec, Acc, MCC, AUC = model_test(model, X_test, Y_test)
		results.append([TP, FP, TN, FN, Sens, Spec, Acc, MCC, AUC])
		i+=1
		
		del X_train
		del X_test
		del Y_train
		del Y_test
		gc.collect()
		
	mean_results = np.mean(results, axis=0)
	print(f'TP={mean_results[0]:.4}, FP={mean_results[1]:.4}, TN={mean_results[2]:.4}, FN={mean_results[3]:.4}, Sens={mean_results[4]:.4}, Spec={mean_results[5]:.4}, Acc={mean_results[6]:.4}, MCC={mean_results[7]:.4}, AUC={mean_results[8]:.4}\n')
	write_data.append(mean_results[0])
	write_data.append(mean_results[1])
	write_data.append(mean_results[2])
	write_data.append(mean_results[3])
	write_data.append(mean_results[4])
	write_data.append(mean_results[5])
	write_data.append(mean_results[6])
	write_data.append(mean_results[7])
	write_data.append(mean_results[8])



In [ ]:
if(VALIDATION_MODE=="independent"):
	if IMBALANCE!="None":
		x_train,y_train=IMBALANCE_HANDLING(IMBALANCE,x_train,y_train)
    #
	time_log("Start Model Train")
	model = DeepScan(
		num_filters=NUM_FILTER,
		num_hidden=NUM_HIDDEN,
		window_sizes=WINDOW_SIZES)
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
	model.build(input_shape=x_train.shape)
	model.summary()

	model.fit(
		x_train,
		y_train,
		batch_size=BATCH_SIZE,
		epochs=EPOCHS,
		shuffle=True,
	)
	time_log("End Model Train")
	time_log("Start Model Test")
	TP,FP,TN,FN,Sens,Spec,Acc,MCC,AUC = model_test(model, x_test, y_test)
	write_data.append(TP)
	write_data.append(FP)
	write_data.append(TN)
	write_data.append(FN)
	write_data.append(Sens)
	write_data.append(Spec)
	write_data.append(Acc)
	write_data.append(MCC)
	write_data.append(AUC)
    
	time_log("End Model Test")

In [ ]:
def save_csv(write_data,a):
    import csv
    b=datetime.datetime.now()
    write_data.append(b-a)
    open_csv=open("imbalance_real.csv","a")
    write_csv=csv.writer(open_csv)
    write_csv.writerow(write_data)
save_csv(write_data,a)